<a href="https://colab.research.google.com/github/kikiru328/All_about_Study/blob/main/EHWA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/골연령예측사진20.zip','/content/Sample')
shutil.unpack_archive('/content/drive/MyDrive/boneage_program.zip','/content/Bone')

In [2]:
import sys
sys.path.append('/content/Bone/boneage')
import NEW_BONE as bone

In [13]:
os.path.basename(path).replace('bmp','jpg')

'97028583.jpg'

In [66]:
path ='/content/Sample/남/10431539.bmp'

In [95]:
import pandas as pd
import datetime as dt
data = pd.read_excel('/content/Sample/골연령측정자료220307.xlsx')
data = data.dropna()
data['Cage'] = data['Cage'].round(1)
data['Patient #'] = data['Patient #'].astype(int).astype(str)
patient_id = os.path.basename(path)[:-4]
display(data)
index = data[data['Patient #'] == patient_id].index[0]
# age = ((data['촬영일자']-data['생년월일'])/365)[1].days
gender = data[(data['Patient #'] == patient_id)]['sex'][index]
he = data[(data['Patient #'] == patient_id)]['촬영일키'][index]
year = data[(data['Patient #'] == patient_id)]['Cage'][index]

,No,sex,생년월일,촬영일자,Cage,촬영일키,Patient #
1,103.0,M,1992-09-07,2004-05-19,11.7,138.0,97028583
2,106.0,M,1992-09-08,2004-08-04,11.9,143.0,10073987
3,101.0,M,1992-08-06,2003-09-17,11.1,137.5,96248548
4,107.0,M,1991-10-01,2003-09-17,12.0,135.8,10431539
5,111.0,M,1990-03-16,2002-09-24,12.5,149.9,96579259
6,115.0,M,1990-10-04,2003-07-23,12.8,150.7,10181949
7,117.0,M,1991-09-12,2004-11-17,13.2,146.1,96178702
8,121.0,M,1990-06-07,2004-01-28,13.6,167.0,10406334
9,124.0,M,1992-10-27,2006-10-25,14.0,152.5,10353946
10,127.0,M,1992-01-03,2006-11-29,14.9,160.3,10580752


'M'

In [5]:
path = '/content/Sample/남/97028583.bmp'

import sys
import os
import cv2
sys.path.append('/content/Bone/boneage')
import NEW_BONE as bone

def basic(path):
    import os
    import pandas as pd
    import datetime as dt
    data = pd.read_excel('/content/Sample/골연령측정자료220307.xlsx')
    data = data.dropna()
    data['Cage'] = data['Cage'].round(1)
    data['Patient #'] = data['Patient #'].astype(int).astype(str)
    patient_id = os.path.basename(path)[:-4]
    index = data[data['Patient #'] == patient_id].index[0]
    gender = data[(data['Patient #'] == patient_id)]['sex'][index]
    if 'M' in gender: 
        gender = 1 
    else: gender = 2

    he = data[(data['Patient #'] == patient_id)]['촬영일키'][index]
    year = data[(data['Patient #'] == patient_id)]['Cage'][index]

    import matplotlib.pyplot as plt
    import cv2
    import os
    import pandas as pd
    import torch
    import tensorflow.keras as tf

    model_path = '/content/Bone/boneage/weight/model.pt'
    tjnet_path = '/content/Bone/boneage/weight/tjnet24.h5'
    lms_df = pd.read_csv('/content/Bone/boneage/data/height_df.csv')
    global tjnet
    tjnet = tf.models.load_model(tjnet_path, compile=False)

    global yolo
    yolo = torch.load(model_path, map_location='cpu')
    return gender,he,year,tjnet,yolo


gender,he,year,tjnet,yolo = basic(path)


file_name = os.path.basename(path).replace('bmp','jpg')

jpg_path = '/content/jpg/'
processed_img = bone.Bone_extraction(path)

cv2.imwrite(jpg_path+file_name,processed_img)

crops, yoloimg, result = bone.yolo_crop_img(jpg_path+file_name,yolo)








# out_crop_img 에서 오류남



X = bone.out_crop_img(crops,gender)
global prediction_BA
prediction_BA = bone.predict_zscore(X, tjnet)
prediction_BA = prediction_BA.round(2)
prediction_H = bone.Height_prediction(gender,prediction_BA,he,lms_df)

print(prediction_BA)
print(prediction_H)

IndexError: ignored

In [ ]:


import torch
import tensorflow.keras as tf
model_path = './weight/model.pt'
tjnet_path = './weight/tjnet24.h5'

global tjnet
tjnet = tf.models.load_model(tjnet_path, compile=False)

global yolo
yolo = torch.load(model_path, map_location='cpu')
